# Práctica 3 - Redes Neuronales Residuales 

### Natalia Martínez García, Lucía Vega Navarrete
### Grupo: AP.11.06

In [19]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### 1. Carga y preprocesado del dataset 

#### Carga 

In [28]:
DATA_PATH = "C:/Users/NataliaUDC/Downloads/higgs/HIGGS.csv.gz"

# Cargar el dataset 
higgs = pd.read_csv(DATA_PATH, compression="gzip", header=None)

print("Tamaño total del dataset (ejemplos, columnas):", higgs.shape)

# Separar características (X) y etiquetas (y)
#   - Columna 0: label
#   - Columnas 1-28: features
X = higgs.iloc[:, 1:].to_numpy(dtype=np.float32) 
y = higgs.iloc[:, 0].to_numpy(dtype=np.float32)

# División según especificaciones: train inicial, test final, extra intermedio
N_TRAIN = 2000000
N_TEST = 500000

X_train = X[:N_TRAIN]
y_train = y[:N_TRAIN]

X_test  = X[-N_TEST:]
y_test  = y[-N_TEST:]

X_extra = X[N_TRAIN:-N_TEST]
y_extra = y[N_TRAIN:-N_TEST]

print("División del dataset:")
print(f"Train: {X_train.shape}")
print(f"Test:  {X_test.shape}")
print(f"Extra: {X_extra.shape}")

Tamaño total del dataset (ejemplos, columnas): (11000000, 29)
División del dataset:
Train: (2000000, 28)
Test:  (500000, 28)
Extra: (8500000, 28)


#### Preprocesado

In [17]:
# Normalizar usando solo TRAIN
# Media y desviación por columna (feature) 
#    (x - media_train) / std_train

"""
Normalizamos solo en train porque cualquier información del test no puede influir en el entrenamiento (el modelo solo puede ver estadísticas del train).
Si usamos la media y desviación del test para normalizar, meteríamos información del futuro dentro del entrenamiento.

Eso se llama data leakage (filtración de información) y hace que tus resultados no sean realistas.
"""
mean_train = X_train.mean(axis=0) # axis = 0 para calcularlo por columnas (cada feature)
std_train  = X_train.std(axis=0)

# Evitar división por cero
std_train[std_train == 0] = 1.0

# Aplicar la transformación estándar: (x - media) / sd
X_train = (X_train - mean_train) / std_train
X_extra = (X_extra - mean_train) / std_train
X_test  = (X_test  - mean_train) / std_train

print("\n Normalización completada")
print(f"  - Media global train (ya normalizado): {X_train.values.mean():.5f}")
print(f"  - Std global train  (ya normalizado): {X_train.values.std():.5f}")

# Comprobación del desbalanceo de clases
def class_stats(name, y):
    n = len(y)
    n_signal = np.sum(y == 1)
    n_back   = np.sum(y == 0)
    print(f"\n{name}:")
    print(f"  Signal     = {n_signal} ({100*n_signal/n:.2f}%)")
    print(f"  Background = {n_back}   ({100*n_back/n:.2f}%)")

print("\n Distribución de clases:")
class_stats("Train", y_train)
class_stats("Extra", y_extra)
class_stats("Test",  y_test)



 Normalización completada
  - Media global train (ya normalizado): 0.00000
  - Std global train  (ya normalizado): 1.00000

 Distribución de clases:

Train:
  Signal     = 1058818 (52.94%)
  Background = 941182   (47.06%)

Extra:
  Signal     = 4505798 (53.01%)
  Background = 3994202   (46.99%)

Test:
  Signal     = 264507 (52.90%)
  Background = 235493   (47.10%)


### 2. Creación de red neuronal con capas residuales

In [27]:
input_dim = X_train.shape[1]  
print("Tamaño de entrada:", input_dim)

def residual_block(x, units, name):
    """
    Bloque residual con:
      - 2 capas lineales (Dense) internas
      - 1 conexión residual (skip connection)
    """
    x_skip = x  # conexión residual

    # 2 capas lineales antes de hacer la suma residual
    x = layers.Dense(units, activation="relu", name=f"{name}_dense1")(x)
    x = layers.Dense(units, activation="relu", name=f"{name}_dense2")(x)

    x = layers.Add(name=f"{name}_add")([x, x_skip]) # suma residual
    return x

inputs = keras.Input(shape=(input_dim,), name="higgs_input") 
x = layers.Dense(128, activation="relu", name="dense_in")(inputs) # Capa de entrada (proyección a dimensión oculta)
x = residual_block(x, units=128, name="res_block1") # Bloque residual 1 
x = residual_block(x, units=128, name="res_block2") # Bloque residual 2 
x = layers.Dense(64, activation="relu", name="dense_mid")(x) # (Opcional) capa intermedia antes de la salida
outputs = layers.Dense(1, activation="sigmoid", name="output")(x) # Capa de salida: 1 neurona con sigmoid para clasificación binaria

model = keras.Model(inputs=inputs, outputs=outputs, name="higgs_resnet") # Definir el modelo

model.summary() # Mostrar resumen para comprobar dimensiones

Tamaño de entrada: 28


Model: "higgs_resnet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ higgs_input         │ (None, 28)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_in (Dense)    │ (None, 128)       │      3,712 │ higgs_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_block1_dense1   │ (None, 128)       │     16,512 │ dense_in[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_block1_dense2   │ (None, 128)       │     16,512 │ res_block1_dense… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_block1_add      │ (None, 128)       │          0 │ res_block1_dense… │
│ (Add)               │                   │            │ dense_in[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_block2_dense1   │ (None, 128)       │     16,512 │ res_block1_add[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_block2_dense2   │ (None, 128)       │     16,512 │ res_block2_dense… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_block2_add      │ (None, 128)       │          0 │ res_block2_dense… │
│ (Add)               │                   │            │ res_block1_add[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_mid (Dense)   │ (None, 64)        │      8,256 │ res_block2_add[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         65 │ dense_mid[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 78,081 (305.00 KB)

 Trainable params: 78,081 (305.00 KB)

 Non-trainable params: 0 (0.00 B)

#### 3. Entrenamiento de la red